In [1]:
import xmltodict
import pandas as pd
import os
"""
	El código toma el archivo recibido de palo alto como xml y lo 
	convierte a un archivo JSON que puede ser leído de manera más 
	sencilla por el programa de mapa de ataques o cualquier otro que
	consuma la info por JS
"""

'\n\tEl código toma el archivo recibido de palo alto como xml y lo \n\tconvierte a un archivo JSON que puede ser leído de manera más \n\tsencilla por el programa de mapa de ataques o cualquier otro que\n\tconsuma la info por JS\n'

In [2]:
home = os.path.expanduser('~')

color_code = {'critical':'red',
              'high':'orange',
             'medium':'yellow',
             'low':'green',
             'informational':'blue'}
default_country = 'MX'

In [3]:
with open('{}/pewpew/XML/LastHour.xml'.format(home)) as fd:
	doc = xmltodict.parse(fd.read())

reform = {(outerKey, innerKey): values for outerKey, innerDict in doc.items() for innerKey, values in innerDict.items()}
a = pd.DataFrame(reform)
reform = a['response']['result']['log']
a = pd.DataFrame(reform)
total = a['logs']['@count']
reform = a['logs']['entry']
df = pd.DataFrame(reform)
#df = df.drop_duplicates(subset=['threatid','src']).reset_index(drop=True)

In [4]:
df = df[['direction','device_name','time_generated',"src",'srcloc','dst','dstloc','subtype','threatid','severity']]
df1 = df['device_name'].map(lambda x : x.split('-')[1])

In [5]:
countries = pd.read_csv(os.path.expanduser('~/pewpew/CSV/all_countries.csv'),sep='\t',index_col=0)
countries['country'] = countries['country'].map(lambda x: x.strip())

In [6]:
Tec = pd.read_csv(os.path.expanduser('~/pewpew/CSV/GPSTec.csv'))

In [7]:
df['srcname'] = ''
df['srclat'] = ''
df['srclong'] = ''
df['dstname'] = ''
df['dstlat'] = ''
df['dstlong'] = ''
a = len(df)
for idx in range(a):
    print('{}/{}'.format(idx+1,a),end='\r')
    
    """
    Pone el codigo alpha-2 como identificador de source y destiny
    """
    df['dstloc'][idx] = df['device_name'][idx].split('-')[1]
    df['srcloc'][idx] = df['srcloc'][idx]['@cc']
    
    """
    Si viene como rango de IP, cambia por el default ('MX')
    
    Checar aqui las ubicaciones del TEC!!!!
    
    """
#     if len(df['dstloc'][idx]) > 2:
#         #print('{}:{}->"MX"'.format(idx,df['dstloc'][idx]))
#         df['dstloc'][idx] = df['device_name'][idx].split('-')[1]
    if len(df['srcloc'][idx]) > 2:
        #print('{}->"MX"'.format(df['srcloc'][idx]))
        df['srcloc'][idx] = default_country ######################################3
    
    """
    Cambia severidad por codigo de colores
    """
    df['severity'][idx] = color_code[df['severity'][idx]]
    
    """
    Cambia el nombre del pais de acuerdo al codigo alpha-2
    """
    df['srcname'][idx] = str(countries[countries['country'] == df['srcloc'][idx]]['name']).split()[1]
    
    tmp = str(Tec[Tec['Campus'] == df['dstloc'][idx]]['Nombre']).split()[1] #!!!!!!!!!!!!!!!!!
    
    if tmp != 'Name:':
        df['dstname'][idx] = tmp
    else:
        df['dstname'][idx] = str(countries[countries['country'] == df['dstloc'][idx]]['name']).split()[1]
    

    """
    TODO
    Cambia cordinadas de destino de acuerdo al nombre del dispositivo
    """
    tmp = str(Tec[Tec['Campus'] == df1.iloc[idx]]['longitud']).split()[1]
    if tmp == 'Name:':
        tmp = str(countries[countries['country'] == df['dstloc'][idx]]['longitude']).split()[1]
    df['dstlong'][idx] = tmp
    tmp = str(Tec[Tec['Campus'] == df1.iloc[idx]]['latitud']).split()[1]
    if tmp == 'Name:':
        tmp = str(countries[countries['country'] == df['dstloc'][idx]]['latitude']).split()[1]
    df['dstlat'][idx] = tmp    
    
    """
    Cambia cordinadas de fuente 
    """
    tmp = str(countries[countries['country'] == df['srcloc'][idx]]['longitude']).split()[1]
    df['srclong'][idx] = tmp
    tmp = str(countries[countries['country'] == df['srcloc'][idx]]['latitude']).split()[1]
    df['srclat'][idx] = tmp    
    

In [8]:
a = ['device_name',
 'direction',
 'src',
 'srclat',
 'srclong',
 'srcloc',
 'srcname',
 'dst',
 'dstlat',
 'dstlong',
 'dstloc',
 'dstname',
 'severity',
 'subtype',
 'threatid',
 'time_generated']

In [9]:
df = df[a]

In [10]:
df

,device_name,direction,src,srclat,srclong,srcloc,srcname,dst,dstlat,dstlong,dstloc,dstname,severity,subtype,threatid,time_generated
0,UTM-FW-IPSEC,client-to-server,10.79.10.112,23.0,-102.0,MX,Mexico,10.73.0.93,25.743988,-100.282816,FW,Firewall,yellow,vulnerability,Microsoft Active Directory DCSync Attempt Dete...,2018/09/27 08:29:27
1,CP-CHI-1,client-to-server,217.61.4.135,51.0,9.0,DE,Germany,10.32.80.248,20.73174,-103.453782,CHI,Chihuahua,orange,vulnerability,Netis/Netcore Router Default Credential Remote...,2018/09/27 08:29:01
2,CP-PUE-1,client-to-server,217.61.4.135,51.0,9.0,DE,Germany,10.50.65.2,19.020044,-98.244838,PUE,Puebla,orange,vulnerability,Netis/Netcore Router Default Credential Remote...,2018/09/27 08:28:58
3,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 08:28:55
4,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 08:28:54
5,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 08:28:52
6,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.230,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 08:28:52
7,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 08:28:49
8,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 08:28:48
9,CP-CEDES-P1,client-to-server,217.61.4.135,51.0,9.0,DE,Germany,131.178.15.132,25.65142,-100.290328,CEDES,CEDES,orange,vulnerability,Netis/Netcore Router Default Credential Remote...,2018/09/27 08:28:44


In [12]:
df.to_json(os.path.expanduser('~/pewpew/XML/_LastHour.json'),orient='index')